 # **Classification de genres musicaux**
 
License : CC BY-NC-SA 4.0

Ce notebook permet d'entraîner un modèle de Deep Learning..

L'objectif est d'entraîner un modèle afin de classifier automatiquement une musique selon son genre musical.

Pour cela, nous allons faire usage d'une base de données (GTZAN) qui va servir à l'entrainement de plusieurs modèles. Nous utiliserons le framework PyTorch ainsi que ses extensions torchaudio/torchvision. Les calculs seront accélérés matériellement par l'usage des GPUs de Jean-Zay.

> Vérifions la présence du ou des GPUs réservés : le notebook doit avoir été lancé depuis un nœud de calcul et non une frontale pour profiter de GPUs.

> La commande "!nvidia-smi" permet de rapidement visualiser l'état des GPUs.

In [ ]:
!module list
!conda env list
!nvidia-smi
from torch.cuda import device_count
print('Torch detecting {0} GPUs compatible with CUDA'.format(device_count()))

## **Imports**

Plusieurs librairies seront utilisées tout au long du TP afin de procéder aux transformations des données, à la gestion de fichiers, à la visualisation, ...

> Vous pouvez modifier les scripts importés et les ré-importer sans relancer le kernel.

In [ ]:
#!module load pytorch-gpu/py3/1.9.0
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np # Maths
import pandas as pd #Data analysis and manipulation
#from sklearn.utils import shuffle

import os # Operating System interaction

import matplotlib # Visualization
import matplotlib.pyplot as plt
#Nécessaire pour visualiser les figures dans le notebook
%matplotlib inline 

from tqdm import tqdm #progress bar

import torch

> Attention : sous windows, il faut installer soundfile (pip install soundfile) et ajouter la ligne suivante :

> torchaudio.set_audio_backend("sox_io")

In [ ]:
# Fixe l'aléatoire pour la reproductibilité
np.random.seed(0)
torch.manual_seed(0)

# Quelques variables pour la gestion des fichiers
project_dir = os.getcwd()
data_dir = '/mnt/datasets/TP_Methodologie/GTZAN/data'
figures_dir = os.path.join(project_dir, 'figures')
scripts_dir = os.path.join(project_dir, 'scripts')
models_dir = os.path.join(project_dir, 'models')
dataset_dir = os.path.join(data_dir, 'raw')

---
---
---
---
---
## **Séparation des données**


Afin d'obtenir un modèle capable de déterminer le genre d'un extrait musical, il est nécessaire de séparer la base de données en plusieurs parties ne partageant pas d'extraits afin d'éviter l'entraînement d'un modèle trop spécialisé. Le résultat de l'évaluation sera biaisé si des informations fuient de la base d'entraînement vers la base de test ou si ces données sont trop corrélées.

Une fois satisfait par un modèle, on peut alors considérer utiliser l'ensemble du dataset pour améliorer encore un peu plus les performances ou du moins réduire la portion dédiée à l'évaluation.

> Nous pouvons utiliser le split standard proposé par torchaudio ce qui peut être utile pour comparer nos performances facilement avec d'autres personnes autour d'une tâche standardisée. Nous pouvons aussi définir une séparation manuellement facilitée par la classe torch.utils.data.Dataset et torch.utils.data.random_split.

> Dans cet exercice, le set valid permet d'évaluer si la loss du modèle diminue correctement. Dans le cas inverse, nous stoppons l'entraînement (early stopping).

> **N'hésitez pas à modifier la séparation des données pour vos différents entraînements. Pour cela, utilisez les paramètres "train_ratio", "valid_ratio" et "test_ratio"**

> Rappel : Les données d'entraînement et de test ne doivent pas être partagées.

In [ ]:
from torch import nn
from scripts.utils import load_gtzan_sets, crop_audio
import torchaudio.transforms as tat
import torchvision.transforms as tvt
transformation = tvt.Compose([tvt.Lambda(crop_audio)])
train_set, valid_set, test_set = load_gtzan_sets(root = dataset_dir, filtered = True, generator = 2022, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, transform = transformation)
print(' Train size : {} \n Valid size : {} \n Test_size : {}'.format(len(train_set), len(valid_set), len(test_set)))

---
---
---
---
---
## **Transformation et Augmentation des données**

Il est possible d'apporter des transformations aux données pour mettre en avant certaines caractéristiques, faire correspondre les données avec celles pour lesquelles un modèle a montré les meilleures performances ou encore améliorer l'entraînement.

Il est souvent recommandé d'effectuer de l'augmentation de données afin de rendre le modèle plus robuste face à certains effets rare dans le dataset, mais que l'on peut générer artificiellement.

Dans le cas de la musique, bien que l'on ait généré des images, l'exercice reste très différent de la classification de photos d'animaux. En effet, les changements d'orientation et les déformations géométrique n'ont pas de sens pour un spectrogramme.

L'ensemble des transformations sont regroupées dans un module (tvt.Compose) qui a l'appel d'une donnée applique la séquence de transformations à la donnée.

Les transformations ne sont pas faites en amont lors de la déclaration du dataset mais lors de l'appel des données. Ainsi chaque epoch est différente à condition d'avoir une probabilité sur l'application ou non d'une transformation ainsi qu'une "seed" différente à chaque epoch. C'est de cette manière que chaque epoch fait usage de données différentes et qu'on obtient une augmentation artificielle de la quantité de données disponibles.

> **Toutes les transformations ne sont pas utiles. Il est bien sûr possible de les essayer, mais une réflexion en amont sur la nature des données est plus efficace.**

> **Attention, si la préparation des données doit être appliquée aux jeux de données d'entraînement et de test, l'augmentation ne doit servir qu'à l'entraînement. Définissez deux transformations différentes selon le set concerné en vous inspirant de ce qui a été exploré à la fin du premier notebook.**





In [ ]:
from torch import nn
from scripts.utils import load_gtzan_sets, crop_audio
import torchaudio.transforms as tat
import torchvision.transforms as tvt
from scripts.utils import check_sizes
transformation = tvt.Compose([tvt.Lambda(lambda waveform: crop_audio(waveform, duration = 5, augment=True)),
                              tat.Spectrogram()])
test_transformation = tvt.Compose([tvt.Lambda(lambda waveform: crop_audio(waveform, duration = 5, augment=False)),
                                  tat.Spectrogram()])

train_set, _, _ = load_gtzan_sets(root = dataset_dir, filtered = True, generator = 2022, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, transform = transformation)
_, valid_set, test_set = load_gtzan_sets(root = dataset_dir, filtered = True, generator = 2022, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, transform = test_transformation)

sizes_uniformity, sizes = check_sizes(train_set)

> Assurez vous d'avoir l'ensemble des sorties avec les mêmes dimensions pour un entraînement depuis zéro ou aux dimensions des données ayant servi à l'entraînement initial pour du transfer learning.

> Il est parfois nécessaire de transformer les données pour correspondre à un format attendu en entrée d'un modèle (nombre de canaux d'une image par exemple). Nous avons dans ce TP privilégié la modification du modèle.

---
---
---
---
---
## **Data Loader**

> Le DataLoader est une classe utilitaire permettant de parcourir les bases de données (datasets) suivant une stratégie prédéfinie : taille des batchs, sampling, randomisation, ...
Selon la stratégie choisie, l'entraînement d'un modèle peut varier très fortement.

> **Lors de l'entraînement, il est nécessaire de faire des compromis entre la taille des batchs et l'occupation mémoire, tout en surveillant la convergence de notre modèle vers une solution satisfaisante. Lors de vos entraînements, si vous avez des dépassements de mémoire ou si vous constatez qu'à l'inverse il en reste, vous pouvez venir modifier le paramètre batch_size. Il peut être nécessaire de redémarrer le kernel.**

> **Réduire la taille de batch lors de l'entraînement peut avoir un effet de régularisation.**

In [ ]:
from torch.utils.data import DataLoader

batch_size = 10
train_loader = DataLoader(train_set, batch_size, shuffle=True, num_workers=4, pin_memory=False)
valid_loader = DataLoader(valid_set, 1, num_workers=4, pin_memory=False)
test_loader = DataLoader(test_set, 1, num_workers=4, pin_memory=False)

---
---
---
---
---
## **Définition du modèle**

Les modèles préparés pour cet exercice sont tous disponibles dans "scripts/models.py' ou dans torchvision.models : https://pytorch.org/vision/stable/models.html

> **Vous pouvez tester de changer de modèle afin de comparer les performances. Vous aurez sûrement à changer le nombre de classe et la première couche du modèle qui attend une image RGB (3 channels). En visualisant la structure du model, vous devriez pouvoir déduire comment modifier la première couche pour l'adapter. En commentaires, vous trouverez les modifications nécessaires pour VGG et Resnet.**

 > **Afin de profiter de l'accélaration matérielle permise par le GPU, il est nécessaire de déplacer le model dans la mémoire de celui-ci.**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Currently used device is :', device)

In [ ]:
from scripts.models import *
import torchvision.models as models

model = models.vgg16(num_classes=10)
model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

#model = models.resnet18(num_classes=10)
#model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

model.to(device)

> Si plusieurs GPU sont disponibles, il est possible d'utiliser des stratégies de parallélisation afin d'accélérer l'entraînement. Plusieurs stratégies de parallélisation sont utilisables. La plus facile à implémenter est le Data Parallelism : http://www.idris.fr/jean-zay/gpu/jean-zay-gpu-torch-multi.html

---
---
---
---
---
## **Entrainement du modèle**

> **En plus du modèle, il est aussi possible de modifier les paramètres d'entraînement du modèle (hyper-paramètres) : optimizer, learning rate, batch size, nombre d'epochs,...**

> Le choix de l'optimizer a un impact sur la convergence du modèle. **Plusieurs optimiseurs sont disponibles :https://pytorch.org/docs/1.11/optim.html**

> Vous ne devriez pas avoir à changer le critère d'évaluation de la loss malgré que d'autres critères existent pour la classification.

In [ ]:
start_epoch = 1
nbr_epochs = 100
epoch_interval = 1
batch_interval = 10
verbose = True
history = None

### **Lorsque vous relancez un entraînement, faites-le depuis la cellule ci-dessous. Si vous changez de modèle et de données, faites-le depuis la cellule au-dessus.**

In [ ]:
from torch import optim
lr = 0.001
momentum = 0.9
weight_decay = 0.0001
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
#optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

> **Vous pouvez interrompre l'entraînement pendant celui-ci en utilisant le bouton stop de jupyterlab. Les sorties seront tout de même retournées.**

> **Cela vous permet d'adapter les hyper-paramètres pendant l'entraînement. Attention à ne pas réinitialiser le compteur d'epoch et history.**

In [ ]:
import time
from scripts.utils import train
experiment_name = str(type(model))[:-2].split('.')[-1] + '_' +str(int(time.time()))


model, history, start_epoch = train(train_loader, 
                                valid_loader, 
                                model,
                                optimizer, 
                                criterion,
                                start_epoch,
                                nbr_epochs,
                                device, 
                                epoch_interval,
                                batch_interval,
                                verbose,
                                models_dir,
                                experiment_name,
                                history)

## **Evaluation du modèle**

L'évaluation finale du modèle doit être fait sur des données n'ayant pas été utilisées pour l'entraînement de celui-ci.
Nous évaluons les meilleures performances du modèle. Pour cela, lors de chaque entraînement, le modèle a été automatiquement sauvegardé dans le dossier models lorsqu'un gain de performance a été constaté.
> **Allez dans le dossier models et copiez le nom du modèle que vous souhaitez évaluer dans la cellule ci-dessous.**

In [ ]:
from scripts.utils import test
model_name = "VGG_1707212218_val_1.73483.t7" 
model_path = os.path.join(models_dir, model_name)
model = torch.load(model_path)
test_loss, predictions, targets = test(test_loader, model, criterion, device)

### **Evolution de la fonction de coût**

En observant la loss, on peut déterminer la qualité de l'apprentissage.
On peut en déduire :
- une convergence trop lente nécessitant d'augmenter learning rate/momentum
- une instabilité qui à l'inverse nécessite peut-être de réduire le learning rate
- du sur-apprentissage
- du sous-apprentissage
- des différences trop grandes entre les datasets qui ne permettent pas de généraliser l'information apprise du dataset d'entraînement à celui de validation
- une mauvaise représentativité du dataset d'évaluation

In [ ]:
from scripts.utils import plot_loss
plot_loss(history['loss'],history['val_loss'])

### **Evolution du taux de réussite**

Le taux de réussite donne une information directe permettant d'évaluer l'accomplissement de la tâche souhaitée.

Elle est cependant rarement auto-suffisante car elle ne montre pas des biais de classification liés à des disparités entre les classes par exemple.

> D'autres métriques existent, choisir la bonne métrique est primordial pour ne pas avoir de déconvenues lors de l'utilisation du modèle sur de nouvelles données par exemple.

In [ ]:
from scripts.utils import plot_accuracy
plot_accuracy(history['acc'],history['val_acc'])

### **Matrice de confusion**

Il existe par exemple la matrice de confusion qui pour des tâches de classification permet de déterminer la qualité du classifieur par classe.

> **Il est préférable de chercher à normaliser les métriques de sorte d'avoir une échelle absolue. La matrice de confusion peut par exemple être normalisée par rapport au nombre d'éléments par classe.**

In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

gtzan_genres = [
    "blues",
    "classical",
    "country",
    "disco",
    "hiphop",
    "jazz",
    "metal",
    "pop",
    "reggae",
    "rock",
]
cm = sklearn.metrics.confusion_matrix(targets, predictions, normalize='true')
df_cm = pd.DataFrame(cm, index = [i for i in gtzan_genres],
                  columns = [i for i in gtzan_genres])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

---
---
---
---
---
## **Sauvegarde du modèle**

> **N'oubliez pas de sauvegarder le modèle dans un fichier, vous ne voudriez pas refaire tout l'entraînement. Il est de bonne pratique de sauvegarder le modèle pendant l'entraînement de celui-ci pour le reprendre à partir d'un point de sauvegarde.**

In [ ]:
model_type = 
model_size =
nbr_epochs =
extra_info =
model_name = model_type + '_' + model_size + '_ '+ nbr_epochs + '_' + extra_info
torch.save(model, os.path.join(models_dir,model_name))

---
---
---
---
---